# Resumo
Neste notebook será criado um modelo para identificar o idioma de textos

# 1. Imports

In [15]:
import pandas as pd
import re
import nlp_functions

from sklearn.model_selection import train_test_split

from nltk.util import bigrams
from nltk.lm.preprocessing import pad_both_ends
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE
from nltk.lm import Laplace

# 2. Carregando dados

In [18]:
SEED = 123

In [19]:
df_port = pd.read_csv('data/stackoverflow_portugues.csv')
df_port.head()

,Id,Título,Questão,Tags,Pontuação,Visualizações
0,2402,Como fazer hash de senhas de forma segura?,"<p>Se eu fizer o <em><a href=""http://pt.wikipe...",<hash><segurança><senhas><criptografia>,350,22367
1,6441,Qual é a diferença entre INNER JOIN e OUTER JOIN?,<p>Qual é a diferença entre <code>INNER JOIN</...,<sql><join>,276,176953
2,579,Por que não devemos usar funções do tipo mysql_*?,<p>Uma dúvida muito comum é por que devemos pa...,<php><mysql>,226,9761
3,2539,As mensagens de erro devem se desculpar?,<p>É comum encontrar uma mensagem de erro que ...,<aplicação-web><gui><console><ux>,214,5075
4,17501,"Qual é a diferença de API, biblioteca e Framew...",<p>Me parecem termos muito próximos e eventual...,<api><framework><terminologia><biblioteca>,193,54191


In [20]:
df_ingles = pd.read_csv('data/stackoverflow_ingles.csv')
df_ingles.head()

,Id,Título,Questão,Tags,Pontuação,Visualizações
0,11227809,Why is it faster to process a sorted array tha...,<p>Here is a piece of C++ code that seems very...,<java><c++><performance><optimization><branch-...,23057,1358574
1,927358,How do I undo the most recent local commits in...,<p>I accidentally committed the wrong files to...,<git><version-control><git-commit><undo>,19640,7906137
2,2003505,How do I delete a Git branch locally and remot...,<p>I want to delete a branch both locally and ...,<git><git-branch><git-remote>,15249,6940906
3,292357,What is the difference between 'git pull' and ...,<blockquote>\n <p><strong>Moderator Note:</st...,<git><git-pull><git-fetch>,11008,2543052
4,477816,What is the correct JSON content type?,"<p>I've been messing around with <a href=""http...",<json><http-headers><content-type>,9701,2478940


# 3. Funções

In [32]:
def remover(textos, regex):
    """
    Remove padrões correspondentes à expressão regular 'regex' dos textos fornecidos.

    Args:
        textos (str ou list): Texto ou lista de textos nos quais aplicar a remoção.
        regex (re.Pattern): Expressão regular compilada para os padrões a serem removidos.

    Returns:
        str ou list: Texto ou lista de textos com os padrões correspondentes removidos.
    """
    if type(textos) == str:
        return regex.sub("", textos)
    else:
        return [regex.sub("", texto) for texto in textos]
    
def substituir_codigo(textos, regex):
    """
    Substitui padrões correspondentes à expressão regular 'regex' por "CODE" nos textos fornecidos.

    Args:
        textos (str ou list): Texto ou lista de textos nos quais aplicar a substituição.
        regex (re.Pattern): Expressão regular compilada para os padrões a serem substituídos.

    Returns:
        str ou list: Texto ou lista de textos com os padrões correspondentes substituídos por "CODE".
    """
    if type(textos) == str:
        return regex.sub("CODE", textos)
    else:
        return [regex.sub("CODE", texto) for texto in textos]
    
def minusculo(textos):
    """
    Converte o texto ou lista de textos para minúsculas.

    Args:
        textos (str ou list): Texto ou lista de textos a serem convertidos.

    Returns:
        str ou list: Texto ou lista de textos em letras minúsculas.
    """
    if type(textos) == str:
        return textos.lower()
    else:
        return [texto.lower() for texto in textos]
    
def substituir_por_espaco(textos, regex):
    """
    Substitui padrões correspondentes à expressão regular 'regex' por espaços nos textos fornecidos.

    Args:
        textos (str ou list): Texto ou lista de textos nos quais aplicar a substituição.
        regex (re.Pattern): Expressão regular compilada para os padrões a serem substituídos por espaços.

    Returns:
        str ou list: Texto ou lista de textos com os padrões correspondentes substituídos por espaços.
    """
    if type(textos) == str:
        return regex.sub(" ", textos)
    else:
        return [regex.sub(" ", texto) for texto in textos]

In [33]:
def calcular_perplexidade(modelo, texto):
    """
    Calcula a perplexidade do modelo de linguagem para o texto fornecido.

    Args:
        modelo: Modelo de linguagem treinado.
        texto (str): Texto para o qual calcular a perplexidade.

    Returns:
        float: Valor da perplexidade do texto de acordo com o modelo de linguagem.
    """
    
    perplexidade = 0
    palavras = nlp_functions.tokenize_whitespace(texto)
    palavras_fakechar = [list(pad_both_ends(palavra, n = 2)) for palavra in palavras]
    palavras_bigramns = [list(bigrams(palavra)) for palavra in palavras_fakechar]
    
    for palavra in palavras_bigramns:
        perplexidade += modelo.perplexity(palavra)
    
    return perplexidade

In [34]:
def treinar_modelo_mle(lista_textos):
    """
    Treina um modelo de linguagem MLE (Maximum Likelihood Estimation) com base nos textos fornecidos.

    Args:
        lista_textos (list): Lista de textos para treinar o modelo.

    Returns:
        MLE: Modelo de linguagem treinado com a abordagem MLE.
    """
    todas_questoes = ' '.join(lista_textos)
    todas_palavras = nlp_functions.tokenize_whitespace(todas_questoes)
    bigrams, vocabulario = padded_everygram_pipeline(2, todas_palavras)
    modelo = MLE(2)
    modelo.fit(bigrams, vocabulario)
    
    return modelo
   

def treinar_modelo_Laplace(lista_textos):
    """
    Treina um modelo de linguagem Laplace com base nos textos fornecidos.

    Args:
        lista_textos (list): Lista de textos para treinar o modelo.

    Returns:
        Laplace: Modelo de linguagem treinado com a abordagem Laplace.
    """
    todas_questoes = ' '.join(lista_textos)
    todas_palavras = nlp_functions.tokenize_whitespace(todas_questoes)
    bigrams, vocabulario = padded_everygram_pipeline(2, todas_palavras)
    modelo = Laplace(2)
    modelo.fit(bigrams, vocabulario)
    
    return modelo

In [35]:
def atribui_idioma(lista_textos, modelo_port, modelo_ing):
    """
    Atribui um idioma (português ou inglês) a cada texto com base na perplexidade calculada pelos modelos de linguagem.

    Args:
        lista_textos (list): Lista de textos para atribuir idiomas.
        modelo_port: Modelo de linguagem para o idioma português.
        modelo_ing: Modelo de linguagem para o idioma inglês.

    Returns:
        list: Lista de strings indicando o idioma atribuído a cada texto.
    """
    
    idioma = []
    for texto in lista_textos:
        portugues = calcular_perplexidade(modelo_port, texto)
        ingles = calcular_perplexidade(modelo_ing, texto)
        if ingles >= portugues:
            idioma.append("portugues")
        else: 
            idioma.append("ingles")

    return idioma

# 4. Regex para tratamento de texto 

In [25]:
# Regex para identificar símbolos do html
regex_html = re.compile(r"<.*?>")
# Regex para identificar códigos
regex_codigo = re.compile(r"<code>(.|(\n))*?</code>")
# Regex para identificar a pontuação
regex_pontuacao = re.compile(r"[^\w\s]")
# Regex para identificar dígitos
regex_digitos = re.compile(r"\d+")
# Regex para identificar espaços
regex_espaco = re.compile(r" +")
# Regex para identificar quebra de linha
regex_quebra_linha = re.compile(r"(\n)")

In [26]:
questoes_port_sem_code = substituir_codigo(df_port.Questão, regex_codigo)
questoes_port_sem_code_tag = remover(questoes_port_sem_code, regex_html)
questoes_port_sem_pont = remover(questoes_port_sem_code_tag, regex_pontuacao)
questoes_port_sem_pont_minus = minusculo(questoes_port_sem_pont)
questoes_port_sem_pont_minus_dig = remover(questoes_port_sem_pont_minus, regex_digitos)
questoes_port_sem_quebra_linha = substituir_por_espaco(questoes_port_sem_pont_minus_dig, regex_quebra_linha)
questoes_port_sem_espaco_duplicado = substituir_por_espaco(questoes_port_sem_quebra_linha, regex_espaco)

df_port["questoes_tratadas"] = questoes_port_sem_espaco_duplicado
df_port["idioma"] = "port"


In [27]:
questoes_ingles_sem_code = substituir_codigo(df_ingles.Questão, regex_codigo)
questoes_ingles_sem_code_tag = remover(questoes_ingles_sem_code, regex_html)
questoes_ingles_sem_pont = remover(questoes_ingles_sem_code_tag, regex_pontuacao)
questoes_ingles_sem_pont_minus = minusculo(questoes_ingles_sem_pont)
questoes_ingles_sem_pont_minus_dig = remover(questoes_ingles_sem_pont_minus, regex_digitos)
questoes_ingles_sem_quebra_linha = substituir_por_espaco(questoes_ingles_sem_pont_minus_dig, regex_quebra_linha)
questoes_ingles_sem_espaco_duplicado = substituir_por_espaco(questoes_ingles_sem_quebra_linha, regex_espaco)

df_ingles["questoes_tratadas"] = questoes_ingles_sem_espaco_duplicado
df_ingles["idioma"] = "ing"

# 5. Pipeline do Modelo

## 5.1. Treinamento dos modelos

In [28]:
port_treino, port_teste = train_test_split(df_port.questoes_tratadas,
                                          test_size = 0.2,
                                          random_state = SEED)


ing_treino, ing_teste = train_test_split(df_ingles.questoes_tratadas,
                                          test_size = 0.2,
                                          random_state = SEED)

In [51]:
modelo_mle_port = treinar_modelo_mle(port_treino)
modelo_mle_ing = treinar_modelo_mle(ing_treino)
modelo_laplace_port = treinar_modelo_Laplace(port_treino)
modelo_laplace_ing = treinar_modelo_Laplace(ing_treino)

## 5.2. Teste dos modelos MLE

In [52]:
resultados_ingles_mle = atribui_idioma(ing_teste, modelo_mle_port, modelo_mle_ing)
resultados_port_mle = atribui_idioma(port_teste, modelo_mle_port, modelo_mle_ing)


taxa_acertos_ingles = (resultados_ingles_mle.count('ingles')/len(resultados_ingles_mle))*100
taxa_acertos_portugues = (resultados_port_mle.count('portugues')/len(resultados_port_mle))*100

In [53]:
print(f'Taxa de acerto dos textos em português: {taxa_acertos_portugues}')
print(f'Taxa de acerto dos textos em inglês: {taxa_acertos_ingles}')

Taxa de acerto dos textos em português: 100.0
Taxa de acerto dos textos em inglês: 83.0


## 5.3. Teste dos modelos Laplace

In [54]:
resultados_ingles_laplace = atribui_idioma(ing_teste, modelo_laplace_port, modelo_laplace_ing)
resultados_port_laplace = atribui_idioma(port_teste, modelo_laplace_port, modelo_laplace_ing)


taxa_acertos_ingles = (resultados_ingles_laplace.count('ingles')/len(resultados_ingles_laplace))*100
taxa_acertos_portugues = (resultados_port_laplace.count('portugues')/len(resultados_port_laplace))*100

In [50]:
print(f'Taxa de acerto dos textos em português: {taxa_acertos_portugues}')
print(f'Taxa de acerto dos textos em inglês: {taxa_acertos_ingles}')

Taxa de acerto dos textos em português: 100.0
Taxa de acerto dos textos em inglês: 83.0
